In [24]:
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad # todo: actually explore these other types of CAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet18
import numpy as np
from PIL import Image
import torchvision
import torch

model = resnet18(pretrained=True)
target_layers = [model.layer4[-1]]
path = '../../data/GradCAM/cat.jpg'
image = np.array(Image.open(path))
image_float_np = np.float32(image) / 255
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])
image_tensor = torchvision.transforms.ToTensor()(image).unsqueeze(0)
input_tensor = torch.cat([image_tensor, image_tensor], dim=0)
print(input_tensor.shape)
print(image.shape)
# input_tensor = input_tensor.unsqueeze(0)
with GradCAM(model=model, target_layers=target_layers, use_cuda=False) as cam:
    category = 281  # presumably, this is the class cat in imagenet?
    targets = [ClassifierOutputTarget(category), ClassifierOutputTarget(category)]
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
    print(grayscale_cam.shape)
for i in range(grayscale_cam.shape[0]):
    print(image.shape)
    visualization = show_cam_on_image(image/255, grayscale_cam[i, :], use_rgb=True)
    Image.fromarray(visualization).show()
# visualization = show_cam_on_image(image/255, grayscale_cam, use_rgb=True)
# Image.fromarray(visualization)


torch.Size([2, 3, 574, 700])
(574, 700, 3)
(2, 574, 700)
(574, 700, 3)
(574, 700, 3)


In [26]:
import gradcam
import finetune
import importlib
importlib.reload(gradcam)
importlib.reload(finetune)

finetune.NUM_CLASSES = 196
finetune.CLASSIFIER_NAME = 'resnet'
finetune.NUM_EPOCHS = 15
finetune.FEATURE_EXTRACT = False
model = finetune.initialize_model()
model = finetune.finetune_model(model)

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.b

KeyboardInterrupt: 

In [ ]:
gradcam.train_distractor(None,model)